### **1. Gemini 2.5 Pro: 네이티브 오디오 + 이미지 멀티모달 체험 (30분)**

**활동 목표:** 이미지와 음성을 **동시에** 입력받아 상황을 이해하고 답변하는 음성 인식 챗봇을 만듭니다. 이를 통해 Gemini가 단순히 텍스트를 넘어 시청각 정보를 함께 처리하는 '네이티브 멀티모달'을 직접 경험합니다.

**기존 멀티 모달과 네이티브 멀티 모달의 차이**

* 기존 멀티모달: 이미지 인식 모델, 음성 인식 모델, 언어 모델이 각각의 결과를 합치는 방식.

* **네이티브 멀티모달 (Gemini 2.5 Pro):** 하나의 모델이 처음부터 이미지와 오디오 스트림을 함께 받아들여 통합적으로 이해. 마치 우리가 눈으로 보고 귀로 들으며 상황을 파악하는 것과 유사합니다.


**실습하기**

1. **환경 설정:**

* 필요한 Python 라이브러리 설치 (`google-generativeai`, `pyaudio`, `Pillow`).

* Google AI Studio에서 발급받은 API 키 인증.

2. **오디오 및 이미지 입력 함수 제작:**

* 웹캠으로 실시간 이미지를 캡처하는 Python 코드 작성.

* 마이크로 실시간 음성을 녹음하는 Python 코드 작성.


3. **Gemini 2.5 Pro 모델 호출:**

* 캡처된 이미지와 녹음된 음성 데이터를 Gemini 모델에 **동시에** 전달.

* **프롬프트 예시:** "여기 사진 속에 있는 물건 보이지? [이미지 데이터] 이걸로 뭘 할 수 있을지 알려줘. [음성 데이터]"

* 모델이 이미지와 음성 컨텍스트를 종합적으로 이해하고 실시간으로 답변을 생성하는 것을 확인.

4. **결과 확인 및 음성 출력:**

* Gemini가 생성한 텍스트 답변을 화면에 출력.

* (심화) Text-to-Speech(TTS) API를 활용하여 답변을 음성으로 변환해 스피커로 출력.

### **2. Google AI Studio를 활용한 Python API 앱 빌드 (30분)**

**활동 목표:** Google AI Studio의 '코드 가져오기' 기능을 사용하여, 방금 만든 멀티모달 챗봇의 핵심 로직을 재사용 가능한 Python API 코드로 변환하는 방법을 배웁니다.

**Google AI Studio란?**

* Google AI Studio는 단순히 모델을 테스트하는 '놀이터'가 아니라, 아이디어를 실제 코드로 빠르게 전환해주는 '프로토타이핑 툴'입니다. 프롬프트 엔지니어링을 통해 원하는 기능을 구현하고, 즉시 Python, cURL 등 다양한 형태의 코드를 얻을 수 있습니다.

**실습하기**

1. **Google AI Studio 접속:**

* 웹 브라우저에서 [Google AI Studio](https://aistudio.google.com/)에 접속.


2. **프롬프트 작성 및 테스트:**


* '새 프롬프트'를 생성하고, 앞선 실습과 유사한 멀티모달 프롬프트를 작성.

* **시스템 명령어(System Instruction) 설정:** "당신은 사용자가 보여주는 이미지와 음성 질문을 바탕으로 창의적인 아이디어를 제안하는 AI 비서입니다." 와 같이 모델의 역할을 구체적으로 정의.

* 예시 이미지와 텍스트 질문을 입력하며 원하는 결과가 나올 때까지 프롬프트를 수정.


3. **코드 생성 및 추출:**

* 화면 상단의 **`</> 코드 가져오기`** 버튼 클릭.

* Python 탭을 선택하여 생성된 API 호출 코드를 복사.

4. **Python 앱에 통합:**


* Colab 노트북으로 돌아와 복사한 코드를 붙여넣기.

* 이 코드가 어떻게 API 키를 사용하고, 모델을 호출하며, 결과를 파싱하는지 구조를 함께 분석.

* (심화) 이 코드를 기반으로 간단한 Flask 또는 FastAPI 웹 서버의 API 엔드포인트로 만드는 방법을 시연.